## Restart the Notebook
After installing BERTopic, some packages that were already loaded were updated and in order to correctly use them, we should now restart the notebook.

From the Menu:

Runtime → Restart Runtime

# Data
For this example, we use the popular 20 Newsgroups dataset which contains roughly 18000 newsgroups posts

In [32]:
import pandas as pd
import re
import nltk

nltk.download('punkt')

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import unidecode
import numpy as np
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'[^a-zA-Z0-9áéíóúÁÉÍÓÚâêîôûÂÊÎÔÛàèìòùÀÈÌÒÙãõñÃÕÑçÇ\s]+', '', text)
    text = unidecode.unidecode(text)
    return text

stopwords = set(stopwords.words('portuguese'))


df_train = pd.read_excel("/home/rafael/Documentos/FACOM/Douturado/Doutorado/webcrawler/medicamentos.xlsx",index_col=0,dtype=str)
df_train.head()
list_preprossing=[]
for dta in range(df_train.shape[0]):
    # value=str(df_train.iloc[dta,0])
    value=str(df_train.iloc[dta,0])+" "+str(df_train.iloc[dta,1])
    # value=str(df_train.iloc[dta,0])+" "+str(df_train.iloc[dta,1]) +" "+str(df_train.iloc[dta,2])+" "+str(df_train.iloc[dta,3])
    tokens = word_tokenize(preprocess_text(value))
    texto_sem_stopwords = [word for word in tokens if word.lower() not in stopwords]
    texto_sem_stopwords = ' '.join(texto_sem_stopwords)
    list_preprossing.append(texto_sem_stopwords)
list_preprossing=np.array(list_preprossing)
list_preprossing=list_preprossing[list_preprossing!='nan']
print(list_preprossing)

[nltk_data] Downloading package punkt to /home/rafael/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


['cafeina carisoprodol diclofenaco sodico paracetamol trimusk cafeina carisoprodol diclofenaco sodico paracetamol indicado tratamento reumatismo formas inflamatoriodegenerativas agudas cronicas crise aguda gota estados inflamatorios agudos postraumaticos poscirurgicos exacerbacoes agudas artrite reumatoide outras artropatias reumaticas osteoartrites estados agudos reumatismo tecidos extraarticulares quadros lombalgias lombociatalgias trimusk cafeina carisoprodol diclofenaco sodico paracetamol indicado coadjuvante processos inflamatorios graves decorrentes quadros infecciosos'
 'probiotico bb12 bifidobacterium animalis subsp lactis suplemento alimentar bifidobacterium animalis subsp lactis bb12 dsm 15954 solucao gotas contribui saude trato gastrointestinal'
 'enoxaparina sodica versa enoxaparina sodica indicado tratamento trombose oclusao trombo veias profundas ja estabelecida embolia pulmonar oclusao algum vaso pulmao trombo migrou outra veia prevencao tromboses venosas associadas ciru

# **Topic Modeling**

In this example, we will go through the main components of BERTopic and the steps necessary to create a strong topic model. 




## Training

We start by instantiating BERTopic. We set language to `english` since our documents are in the English language. If you would like to use a multi-lingual model, please use `language="multilingual"` instead. 

We will also calculate the topic probabilities. However, this can slow down BERTopic significantly at large amounts of data (>100_000 documents). It is advised to turn this off if you want to speed up the model. 


In [ ]:
from bertopic import BERTopic

topic_model = BERTopic(language="multilingual", calculate_probabilities=True, verbose=True)

In [50]:
topics, probs = topic_model.fit_transform(list_preprossing)

2023-05-16 04:05:47,258 - BERTopic - Transformed documents to Embeddings
2023-05-16 04:05:50,582 - BERTopic - Reduced dimensionality
2023-05-16 04:05:50,680 - BERTopic - Clustered reduced embeddings


**NOTE**: Use `language="multilingual"` to select a model that support 50+ languages.

## Extracting Topics
After fitting our model, we can start by looking at the results. Typically, we look at the most frequent topics first as they best represent the collection of documents. 

In [57]:
freq = topic_model.get_topic_info(); freq.head(5)

,Topic,Count,Name
0,-1,30,-1_inflamacao_tratamento_indicado_processos
1,0,70,0_vitamina_indicado_calcio_cloridrato
2,1,50,1_infeccoes_causadas_trato_infeccao
3,2,44,2_transtorno_tratamento_ansiedade_cloridrato
4,3,33,3_dor_dores_osteoporose_indicado


-1 refers to all outliers and should typically be ignored. Next, let's take a look at a frequent topic that were generated:

In [58]:
topic_model.get_topic(0)  # Select the most frequent topic

[('vitamina', 0.033869805842858466),
 ('indicado', 0.032005190405781175),
 ('calcio', 0.03186063988777495),
 ('cloridrato', 0.03063776195901329),
 ('calcica', 0.02648891303369666),
 ('dieta', 0.025393429401660658),
 ('niveis', 0.02537648055536371),
 ('nao', 0.024046480081952766),
 ('betametasona', 0.02349324455012847),
 ('hipercolesterolemia', 0.0231507991020146)]

**NOTE**: BERTopic is stocastich which mmeans that the topics might differ across runs. This is mostly due to the stocastisch nature of UMAP.

In [ ]:
### Attributes

## Attributes

There are a number of attributes that you can access after having trained your BERTopic model:


| Attribute | Description |
|------------------------|---------------------------------------------------------------------------------------------|
| topics_               | The topics that are generated for each document after training or updating the topic model. |
| probabilities_ | The probabilities that are generated for each document if HDBSCAN is used. |
| topic_sizes_           | The size of each topic                                                                      |
| topic_mapper_          | A class for tracking topics and their mappings anytime they are merged/reduced.             |
| topic_representations_ | The top *n* terms per topic and their respective c-TF-IDF values.                             |
| c_tf_idf_              | The topic-term matrix as calculated through c-TF-IDF.                                       |
| topic_labels_          | The default labels for each topic.                                                          |
| custom_labels_         | Custom labels for each topic as generated through `.set_topic_labels`.                                                               |
| topic_embeddings_      | The embeddings for each topic if `embedding_model` was used.                                                              |
| representative_docs_   | The representative documents for each topic if HDBSCAN is used.                                                |

For example, to access the predicted topics for the first 10 documents, we simply run the following:

In [59]:
topic_model.topics_[:10]

[0, 0, 9, -1, 0, 4, 1, 2, 1, 0]

# **Visualization**
There are several visualization options available in BERTopic, namely the visualization of topics, probabilities and topics over time. Topic modeling is, to a certain extent, quite subjective. Visualizations help understand the topics that were created. 

## Visualize Topics
After having trained our `BERTopic` model, we can iteratively go through perhaps a hundred topic to get a good 
understanding of the topics that were extract. However, that takes quite some time and lacks a global representation. 
Instead, we can visualize the topics that were generated in a way very similar to 
[LDAvis](https://github.com/cpsievert/LDAvis):

In [60]:
topic_model.visualize_topics()

## Visualize Topic Probabilities

The variable `probabilities` that is returned from `transform()` or `fit_transform()` can 
be used to understand how confident BERTopic is that certain topics can be found in a document. 

To visualize the distributions, we simply call:

In [61]:
topic_model.visualize_distribution(probs[200], min_probability=0.015)

ValueError: This visualization cannot be used if you have set `calculate_probabilities` to False as it uses the topic probabilities of all topics. 

## Visualize Topic Hierarchy

The topics that were created can be hierarchically reduced. In order to understand the potential hierarchical structure of the topics, we can use scipy.cluster.hierarchy to create clusters and visualize how they relate to one another. This might help selecting an appropriate nr_topics when reducing the number of topics that you have created.

In [62]:
topic_model.visualize_hierarchy(top_n_topics=50)

## Visualize Terms

We can visualize the selected terms for a few topics by creating bar charts out of the c-TF-IDF scores for each topic representation. Insights can be gained from the relative c-TF-IDF scores between and within topics. Moreover, you can easily compare topic representations to each other.

In [63]:
topic_model.visualize_barchart(top_n_topics=5)

## Visualize Topic Similarity
Having generated topic embeddings, through both c-TF-IDF and embeddings, we can create a similarity matrix by simply applying cosine similarities through those topic embeddings. The result will be a matrix indicating how similar certain topics are to each other.

In [40]:
topic_model.visualize_heatmap(n_clusters=5, width=1000, height=1000)

## Visualize Term Score Decline
Topics are represented by a number of words starting with the best representative word. Each word is represented by a c-TF-IDF score. The higher the score, the more representative a word to the topic is. Since the topic words are sorted by their c-TF-IDF score, the scores slowly decline with each word that is added. At some point adding words to the topic representation only marginally increases the total c-TF-IDF score and would not be beneficial for its representation.

To visualize this effect, we can plot the c-TF-IDF scores for each topic by the term rank of each word. In other words, the position of the words (term rank), where the words with the highest c-TF-IDF score will have a rank of 1, will be put on the x-axis. Whereas the y-axis will be populated by the c-TF-IDF scores. The result is a visualization that shows you the decline of c-TF-IDF score when adding words to the topic representation. It allows you, using the elbow method, the select the best number of words in a topic.


In [41]:
topic_model.visualize_term_rank()

# **Topic Representation**
After having created the topic model, you might not be satisfied with some of the parameters you have chosen. Fortunately, BERTopic allows you to update the topics after they have been created. 

This allows for fine-tuning the model to your specifications and wishes. 

## Update Topics
When you have trained a model and viewed the topics and the words that represent them,
you might not be satisfied with the representation. Perhaps you forgot to remove
stopwords or you want to try out a different `n_gram_range`. We can use the function `update_topics` to update 
the topic representation with new parameters for `c-TF-IDF`: 


In [42]:
topic_model.update_topics(list_preprossing, n_gram_range=(1, 2))

In [43]:
topic_model.get_topic(0)   # We select topic that we viewed before

[('infeccoes', 0.056576192449473456),
 ('causadas', 0.026745972276722024),
 ('trato', 0.022824072182768173),
 ('tratamento', 0.02115281304105715),
 ('incluindo', 0.020490799087102742),
 ('infeccao', 0.020197090142041112),
 ('amoxicilina', 0.01800332089614485),
 ('sensiveis', 0.01735787917772679),
 ('infeccoes trato', 0.017005350536956076),
 ('bacterias', 0.016473422124243596)]

## Topic Reduction
We can also reduce the number of topics after having trained a BERTopic model. The advantage of doing so, 
is that you can decide the number of topics after knowing how many are actually created. It is difficult to 
predict before training your model how many topics that are in your documents and how many will be extracted. 
Instead, we can decide afterwards how many topics seems realistic:





In [44]:
topic_model.reduce_topics(list_preprossing, nr_topics=20)

2023-05-16 04:02:14,203 - BERTopic - Reduced number of topics from 14 to 14


In [45]:
# Access the newly updated topics with:
print(topic_model.topics_)

[6, 0, 1, 6, 12, 3, 0, 2, 0, 1, 7, -1, -1, 3, 1, 2, 5, 1, 0, 2, 0, 5, 5, 12, -1, 2, 9, 10, 2, 6, 0, 3, 0, 11, 4, -1, 3, -1, 3, 5, 0, 0, 3, 0, 3, 0, 2, 3, 10, 7, 0, 11, 0, 1, 9, 8, 9, 12, 2, 4, 0, 12, 11, 3, 9, 6, 3, 0, 4, 8, 1, 0, 1, 9, 1, 12, 6, 0, 4, 4, 1, 4, 6, 0, 2, 1, 7, 5, 2, 10, 2, 5, -1, 6, 12, 11, 0, -1, 5, 2, 4, 10, 4, 1, 0, 0, 5, 2, 1, 1, 1, 0, 5, 0, -1, 6, 0, 0, 2, 9, 7, 4, 7, 7, 3, -1, 2, -1, 2, 2, -1, 6, 5, 0, -1, 9, 3, -1, 5, 11, 10, 1, 0, 12, 2, 1, 0, 0, 1, 12, 1, 9, 0, 3, 7, 3, 0, 1, -1, 3, 7, 6, -1, 4, 0, -1, 2, 0, 0, 1, 0, 2, 1, 2, 7, 10, 0, 11, 3, 0, 12, -1, 5, 0, 4, -1, 1, 7, 6, 9, 1, 3, 2, 0, 2, 2, 6, 0, 6, 2, 3, 0, 9, 2, 0, 0, -1, 11, 8, 4, -1, 4, 1, 5, 10, -1, 1, 0, 5, 0, 0, 9, 1, 0, -1, 8, 9, 1, 0, 12, 11, 2, 12, 9, 2, -1, 8, 7, 11, 2, 7, 2, 10, 5, 0, 3, 8, 8, 5, 11, 0, 8, 8, -1, 2, 0, 2, 8, 11, 8, 0, 2, 1, 1, 0, 10, 2, 0, -1, 6, 8, 2, -1, 4, 0, 3, 2, -1, 8, 3, 0, 1, -1, 12, 4, 1, -1, 6, 3, 3, 1, 1, 6, 3, 5, 10, -1, -1, -1, 0, 0, 6, 2, 1, 10, 1, 3, 1, 2, 9, 5, 

# **Search Topics**
After having trained our model, we can use `find_topics` to search for topics that are similar 
to an input search_term. Here, we are going to be searching for topics that closely relate the 
search term "vehicle". Then, we extract the most similar topic and check the results: 

In [46]:
similar_topics, similarity = topic_model.find_topics("infeccoes", top_n=5); similar_topics

[0, 5, 6, 10, 7]

In [47]:
topic_model.get_topic(2 )

[('transtorno', 0.06289965136623425),
 ('tratamento', 0.043800862196485976),
 ('ansiedade', 0.03468342495386545),
 ('cloridrato', 0.030017811549983377),
 ('tratamento transtorno', 0.028650365055367853),
 ('social', 0.024788096595041047),
 ('paroxetina', 0.024788096595041047),
 ('indicado', 0.024689477906194426),
 ('criancas', 0.024326767314578463),
 ('depressao', 0.023179830498516907)]

# **Model serialization**
The model and its internal settings can easily be saved. Note that the documents and embeddings will not be saved. However, UMAP and HDBSCAN will be saved. 

In [ ]:
# Save model
topic_model.save("my_model")	

In [ ]:
# Load model
my_model = BERTopic.load("my_model")	

# **Embedding Models**
The parameter `embedding_model` takes in a string pointing to a sentence-transformers model, a SentenceTransformer, or a Flair DocumentEmbedding model.

## Sentence-Transformers
You can select any model from sentence-transformers here and pass it through BERTopic with embedding_model:



In [49]:
topic_model = BERTopic(embedding_model="xlm-r-bert-base-nli-stsb-mean-tokens")

Or select a SentenceTransformer model with your own parameters:


In [ ]:
from sentence_transformers import SentenceTransformer

sentence_model = SentenceTransformer("distilbert-base-nli-mean-tokens", device="cpu")
topic_model = BERTopic(embedding_model=sentence_model, verbose=True)

Click [here](https://www.sbert.net/docs/pretrained_models.html) for a list of supported sentence transformers models.  
